In [1]:
import rmgpy
import rmg
from rmgpy.data.rmg import RMGDatabase
from rmgpy import settings
from rmgpy.molecule import Molecule
from rmgpy.species import Species
from rmgpy.thermo.thermoengine import submit
import os
path = os.path.join(settings['database.directory'],'thermo')
database = RMGDatabase()

In [2]:
database.load_thermo(path=path,thermo_libraries=['primaryThermoLibrary','surfaceThermoPt111','thermo_DFT_CCSDTF12_BAC','DFT_QCI_thermo'],
              surface=True)

In [3]:
deutschmann = rmgpy.chemkin.load_chemkin_file('deutschmann_pt_rmg_nocov.txt',
                                thermo_path='deutschmann_pt_thermo.txt',
                                dictionary_path='species_dictionary.txt',
                                read_comments=False)

In [4]:
deutschmann_rxns = deutschmann[1]
species_dict = rmgpy.chemkin.load_species_dictionary('species_dictionary.txt')

In [13]:
deutschmann

([],
 [LibraryReaction(index=1, reactants=[Species(label="H2", molecule=[Molecule(smiles="[H][H]")], molecular_weight=(2.01594,'amu')), Species(label="X", molecule=[Molecule(smiles="[Pt]")], molecular_weight=(0,'amu')), Species(label="X", molecule=[Molecule(smiles="[Pt]")], molecular_weight=(0,'amu'))], products=[Species(label="HX", molecule=[Molecule(smiles="[Pt]")], molecular_weight=(1.00797,'amu')), Species(label="HX", molecule=[Molecule(smiles="[Pt]")], molecular_weight=(1.00797,'amu'))], kinetics=StickingCoefficient(A=0.046, n=0, Ea=(0,'kJ/mol'), T0=(1,'K'), coverage_dependence={}), reversible=False),
  LibraryReaction(index=2, reactants=[Species(label="X", molecule=[Molecule(smiles="[Pt]")], molecular_weight=(0,'amu')), Species(label="X", molecule=[Molecule(smiles="[Pt]")], molecular_weight=(0,'amu')), Species(label="O2", molecule=[Molecule(smiles="[O][O]")], molecular_weight=(31.9988,'amu'))], products=[Species(label="OX", molecule=[Molecule(smiles="O=[Pt]")], molecular_weight=(

In [100]:
for label, species in species_dict.items():
    submit(species)

In [102]:
species_dict['H2'].get_enthalpy

<bound method Species.get_enthalpy of Species(label="H2", thermo=NASA(polynomials=[NASAPolynomial(coeffs=[3.43536,0.00021271,-2.78625e-07,3.40267e-10,-7.76032e-14,-1031.36,-3.90842], Tmin=(100,'K'), Tmax=(1959.08,'K')), NASAPolynomial(coeffs=[2.78817,0.000587641,1.5901e-07,-5.52739e-11,4.34311e-15,-596.144,0.112733], Tmin=(1959.08,'K'), Tmax=(5000,'K'))], Tmin=(100,'K'), Tmax=(5000,'K'), E0=(-8.60349,'kJ/mol'), Cp0=(29.1007,'J/(mol*K)'), CpInf=(37.4151,'J/(mol*K)'), label="""H2""", comment="""Thermo library: primaryThermoLibrary"""), conformer=Conformer(E0=(-8.60349,'kJ/mol'), modes=[]), molecule=[Molecule(smiles="[H][H]")], molecular_weight=(2.01594,'amu'))>

In [103]:
import matplotlib.pyplot as plt

In [ ]:
def thermo_comparison_plots():
    f = plt.figure(figsize=(14,4))
    ax = f.add_subplot(131)
    ax2 = f.add_subplot(132)
    ax3 = f.add_subplot(133)
    ax.plot(T_Deut, Cp_R_Deut2, label='Deutschmann')
    ax.plot(T_RMG2, Cp_R_RMG2, label='RMG generated')

    ax2.plot(T_Deut, H_RT_Deut2, label='Deutschmann')
    ax2.plot(T_RMG2, H_RT_RMG2, label='RMG generated')
    ax3.plot(T_Deut, S_R_Deut2, label='Deutschmann')
    ax3.plot(T_RMG2, S_R_RMG2, label='RMG generated')

    ax.set_xlabel('T (K)')
    ax.set_ylabel('Cp/R')
    ax.legend(loc='best')

    ax2.set_xlabel('T (K)')
    ax2.set_ylabel('H/RT')
    ax2.legend(loc='best')

    ax3.set_xlabel('T (K)')
    ax3.set_ylabel('S/R')
    ax3.legend(loc='best')

    ax2.set_title('For adsorbed HO')

In [97]:
deutschmann_rxns[0].get_enthalpy_of_reaction(298)

Exception: Unable to calculate enthalpy for species 'H2': no thermo or statmech data available.

Exception ignored in: 'rmgpy.reaction.Reaction.get_enthalpy_of_reaction'
Traceback (most recent call last):
  File "rmgpy/species.py", line 495, in rmgpy.species.Species.get_enthalpy
Exception: Unable to calculate enthalpy for species 'H2': no thermo or statmech data available.


0.0

In [6]:
import numpy as np
def NASAfun(coeff, T):
    a1, a2, a3, a4, a5, a6, a7 = coeff
    Cp_R = a1 + a2*T + a3*T**2 + a4*T**3 + a5*T**4
    H_RT = a1 + a2*T/2.0 + a3*T**2/3.0 + a4*T**3/4.0 + a5*T**4/5.0 + a6/T
    S_R  = a1*np.log(T) + a2*T + a3*T**2/2.0 + a4*T**3/3.0 + a5*T**4/4.0 + a7
    return Cp_R, H_RT, S_R